---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [11]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    from sklearn import preprocessing
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.preprocessing import MinMaxScaler
    #train_v0 = pd.read_csv("train.csv", encoding = 'ISO-8859-1', 
    #                         dtype={'zip_code': pd.np.str,
    #                                'non_us_str_code': pd.np.str,
    #                                'grafitti_status': pd.np.str})
    train_v0 = pd.read_csv('train.csv', encoding="latin1",low_memory=False)
    np.random.seed(seed=0)
    #train = pd.read_csv('train.csv', encoding = 'ISO-8859-1', low_memory = False)
    X_test_v0 = pd.read_csv('test.csv', encoding="latin1",low_memory=False)
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    addresses = addresses.set_index('address')
    latlons = latlons.set_index('address')
    lat_lon_ticket_id = pd.merge(addresses, latlons, how='left', left_index=True, right_index=True)
    lat_lon_ticket_id = lat_lon_ticket_id.astype(float)
    lat_lon_ticket_id = lat_lon_ticket_id.set_index('ticket_id').dropna()

    #X_train = train.iloc[:,:-1][X_test.columns]
    #train = train.dropna(axis='columns')
    #train =  train.fillna(train.mean())#pd.DataFrame(train).fillna(0)
    train = train_v0.copy()
    X_test = X_test_v0.copy()
    X_test = X_test.reset_index()
    
    train = pd.DataFrame(train).fillna(0)
    train = train.reset_index()
    ##
     
        #
    #lat_loc
    # = lat_lon_ticket_id.loc
    #
    #X_train = X_train.set_index('ticket_id')
    #X_test = X_test.set_index('ticket_id')
    #X_train['lat'] = lat_lon_ticket_id.loc[X_train.index,'lat']
    #X_test['lat'] = lat_lon_ticket_id.loc[X_test.index,'lat']
    

    # TODO: create a LabelEncoder object and fit it to each feature in X


    # 1. INSTANTIATE
    # encode labels with value between 0 and n_classes-1.
    le = preprocessing.LabelEncoder()
    X_test_v0 = pd.DataFrame(X_test_v0[['violation_code']])#.select_dtypes(include=[object])
    X_test_v0 = X_test_v0.apply(le.fit_transform)
    
    train_v0 = pd.DataFrame(train_v0[['violation_code']])#.select_dtypes(include=[object])
    train_v0 = train_v0.apply(le.fit_transform)
    
    ##
    X_train = train.iloc[:,:-1][X_test.columns].select_dtypes(include=['float64','int64']).drop(['violation_zip_code'], axis = 1)
    #X_train[['violation_code']] = train_v0[['violation_code']]
    #
    X_train = X_train.set_index('ticket_id')
    X_test = X_test.set_index('ticket_id')
    X_train['lat'] = lat_lon_ticket_id.loc[X_train.index,'lat']
    X_train['lon'] = lat_lon_ticket_id.loc[X_train.index,'lon']
    X_train = pd.DataFrame(X_train).fillna(0)
    
    X_test['lat'] = lat_lon_ticket_id.loc[X_test.index,'lat']
    X_test['lon'] = lat_lon_ticket_id.loc[X_test.index,'lon']
    X_test = pd.DataFrame(X_test).fillna(0)
    
    #
    y_train = train.iloc[:,[-1]]
    #X_test = X_test.fillna(X_test.mean()) #pd.DataFrame(X_test).fillna(0)
    #_test = X_test.dropna(axis='columns')
    X_test = pd.DataFrame(X_test).fillna(0)
    X_test = X_test.reset_index()
    X_test  = X_test.select_dtypes(include=['float64','int64']).drop(['non_us_str_code'], axis = 1)
    X_test = X_test.set_index('ticket_id')
    X_train = X_test.set_index('ticket_id')

    #X_test[['violation_code']] = X_test_v0[['violation_code']]
    #X_train = X_train.drop(['inspector_name','violator_name'], axis = 1)
    #X_test = X_test.drop(['inspector_name','violator_name'], axis = 1)

    #
    #min_max_scaler = preprocessing.MinMaxScaler()
    #X_train_minmax = min_max_scaler.fit_transform(X_train)
    #X_test_minmax = min_max_scaler.fit_transform(X_test)

    #y_train = train
    #clf = RandomForestClassifier(n_estimators = 30, random_state = 0).fit(X_train_minmax, y_train)
    clf = RandomForestClassifier(bootstrap = True, n_estimators = 30, random_state = 0).fit(X_train, y_train)
    #clf = GradientBoostingClassifier(random_state = 0).fit(X_train_minmax, y_train)
    #print('Random Forest, , default settings')
    #print('Accuracy of RF classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))

    y_test = clf.predict_proba(X_test)[:, 1]
    results = pd.Series(y_test, index=X_test.index, name='compliance', dtype = "float32")
    #results.index.name = 
    return results#, X_test #X_train_minmax # Your answer here
blight_model()
#X_train, X_test = blight_model()
#X_train.info()
#X_test.info()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


ticket_id
284932    0.166667
285362    0.033333
285361    0.366667
285338    0.400000
285346    0.200000
285345    0.400000
285347    0.166667
285342    0.533333
285530    0.166667
284989    0.066667
285344    0.166667
285343    0.066667
285340    0.100000
285341    0.166667
285349    0.200000
285348    0.400000
284991    0.066667
285532    0.200000
285406    0.033333
285001    0.033333
285006    0.033333
285405    0.033333
285337    0.100000
285496    0.200000
285497    0.433333
285378    0.033333
285589    0.100000
285585    0.266667
285501    0.300000
285581    0.033333
            ...   
376367    0.133333
376366    0.333333
376362    0.200000
376363    0.200000
376365    0.133333
376364    0.333333
376228    0.300000
376265    0.266667
376286    0.666667
376320    0.233333
376314    0.033333
376327    0.633333
376385    0.533333
376435    0.466667
376370    0.633333
376434    0.166667
376459    0.200000
376478    0.033333
376473    0.133333
376484    0.166667
376482    0.133333
37